In [24]:
import pandas as pd
import hopsworks
import CleanInputFunction
import TopicExtractionFunction

In [25]:
# Connect to Hopsworks
project = hopsworks.login()
feature_store = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5270
Connected. Call `.close()` to terminate connection gracefully.


In [26]:
# Read in training data
data = pd.read_csv('data/RoundedLabelled.csv')
data = data.rename(columns={'title': 'title', 'topic': 'subjects', 'stance / attitude towards': 'stance', 'article URL': 'url', 'savedAt': 'saved_at', 'main topic': 'labelled_topic'})
data = data.drop(['query', 'language', 'Clickbaityness/credibility (0 is good, 1 is very bad)', 'labelled_topic'], axis=1) # labelled_topic is dropped as this is only for testing the topic extraction and is not used as training data, Hopsworks requires all columns to be filled which won't always be the case for labelled_topic
data

,title,saved_at,url,stance
0,Fed interest rate decision today: Live updates...,14/12/2022 18:11,https://www.usatoday.com/story/money/economy/2...,2.0
1,Moscow police should hand over control of Idah...,14/12/2022 18:11,https://nypost.com/2022/12/14/police-should-ha...,2.0
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022 18:11,https://www.bbc.com/news/world-africa-63973938,2.0
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022 18:11,https://www.youtube.com/watch?v=PesQLjkAEew,2.0
4,Mortgage demand inches higher as interest rate...,14/12/2022 18:11,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0
...,...,...,...,...
414,These lawmakers probing FTX's collapse got don...,16/12/2022 09:23,https://nypost.com/2022/12/15/these-lawmakers-...,1.0
415,Futures: Nasdaq Breaks Key Level; Here's The S...,16/12/2022 09:23,https://www.investors.com/market-trend/stock-m...,3.0
416,Adobe stock climbs on a more profitable quarte...,16/12/2022 09:23,https://www.cnbc.com/2022/12/15/adobe-adbe-ear...,3.0
417,Netflix stock sinks after reported ad target m...,16/12/2022 09:23,https://finance.yahoo.com/news/netflix-stock-s...,1.0


In [27]:
# Pre-process
data = CleanInputFunction.clean_input(data)
display(data)

,title,saved_at,url,stance,title_stance,title_topic
0,Fed interest rate decision today: Live updates...,14/12/2022 18:11,https://www.usatoday.com/story/money/economy/2...,2.0,fed interest rate decision today live updates ...,fed interest rate decision today live updates ...
1,Moscow police should hand over control of Idah...,14/12/2022 18:11,https://nypost.com/2022/12/14/police-should-ha...,2.0,moscow police should hand over control of idah...,moscow police hand control idaho murder invest...
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022 18:11,https://www.bbc.com/news/world-africa-63973938,2.0,world cup 2022 morocco fans heartbreak as doha...,world cup 2022 morocco fans heartbreak doha fl...
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022 18:11,https://www.youtube.com/watch?v=PesQLjkAEew,2.0,scream 6 official teaser trailer 2023 jenna or...,scream 6 official teaser trailer 2023 jenna or...
4,Mortgage demand inches higher as interest rate...,14/12/2022 18:11,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0,mortgage demand inches higher as interest rate...,mortgage demand inches higher interest rates m...
...,...,...,...,...,...,...
414,These lawmakers probing FTX's collapse got don...,16/12/2022 09:23,https://nypost.com/2022/12/15/these-lawmakers-...,1.0,these lawmakers probing ftx s collapse got don...,lawmakers probing ftx collapse got donations s...
415,Futures: Nasdaq Breaks Key Level; Here's The S...,16/12/2022 09:23,https://www.investors.com/market-trend/stock-m...,3.0,futures nasdaq breaks key level here s the sil...,futures nasdaq breaks key level silver lining ...
416,Adobe stock climbs on a more profitable quarte...,16/12/2022 09:23,https://www.cnbc.com/2022/12/15/adobe-adbe-ear...,3.0,adobe stock climbs on a more profitable quarte...,adobe stock climbs profitable quarter predicte...
417,Netflix stock sinks after reported ad target m...,16/12/2022 09:23,https://finance.yahoo.com/news/netflix-stock-s...,1.0,netflix stock sinks after reported ad target m...,netflix stock sinks reported ad target miss an...


In [28]:
# Add topics
data, _ = TopicExtractionFunction.extract_topics(data)
display(data)

2022-12-28 14:13:42,276 INFO: Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
2022-12-28 14:13:44,510 INFO: Use pytorch device: cpu


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

2022-12-28 14:13:50,522 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2022-12-28 14:13:50,673 INFO: Use pytorch device: cpu


,title,saved_at,url,stance,title_stance,title_topic,predicted_topic
0,Fed interest rate decision today: Live updates...,14/12/2022 18:11,https://www.usatoday.com/story/money/economy/2...,2.0,fed interest rate decision today live updates ...,fed interest rate decision today live updates ...,"inflation, fed, rates"
1,Moscow police should hand over control of Idah...,14/12/2022 18:11,https://nypost.com/2022/12/14/police-should-ha...,2.0,moscow police should hand over control of idah...,moscow police hand control idaho murder invest...,"ukraine, russia, war"
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022 18:11,https://www.bbc.com/news/world-africa-63973938,2.0,world cup 2022 morocco fans heartbreak as doha...,world cup 2022 morocco fans heartbreak doha fl...,"cup, world, doha"
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022 18:11,https://www.youtube.com/watch?v=PesQLjkAEew,2.0,scream 6 official teaser trailer 2023 jenna or...,scream 6 official teaser trailer 2023 jenna or...,"ign, showdown, hunt"
4,Mortgage demand inches higher as interest rate...,14/12/2022 18:11,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0,mortgage demand inches higher as interest rate...,mortgage demand inches higher interest rates m...,"fed, cnbc, economy"
...,...,...,...,...,...,...,...
414,These lawmakers probing FTX's collapse got don...,16/12/2022 09:23,https://nypost.com/2022/12/15/these-lawmakers-...,1.0,these lawmakers probing ftx s collapse got don...,lawmakers probing ftx collapse got donations s...,"bankman, fried, sam"
415,Futures: Nasdaq Breaks Key Level; Here's The S...,16/12/2022 09:23,https://www.investors.com/market-trend/stock-m...,3.0,futures nasdaq breaks key level here s the sil...,futures nasdaq breaks key level silver lining ...,"investor, business, electric"
416,Adobe stock climbs on a more profitable quarte...,16/12/2022 09:23,https://www.cnbc.com/2022/12/15/adobe-adbe-ear...,3.0,adobe stock climbs on a more profitable quarte...,adobe stock climbs profitable quarter predicte...,"stock, cramer, cnbc"
417,Netflix stock sinks after reported ad target m...,16/12/2022 09:23,https://finance.yahoo.com/news/netflix-stock-s...,1.0,netflix stock sinks after reported ad target m...,netflix stock sinks reported ad target miss an...,"stock, cramer, cnbc"


In [29]:
# Save result to feature_store
article_cleaned_feature_store = feature_store.get_or_create_feature_group(
    name="training_data_stance",
    version=1,
    primary_key=["url"],
    description="Articles with predicted topic")
article_cleaned_feature_store.insert(data, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/419 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5270/jobs/named/training_data_stance_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1602df426a0>, None)